In [5]:
import tensorflow 

import pandas as pd
import numpy as np
import os
import keras
import random
import cv2
import math
import seaborn as sns

from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

from keras.utils.np_utils import to_categorical
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Convolution2D,BatchNormalization,Activation
from tensorflow.keras.layers import Flatten,MaxPooling2D,Dropout
from tensorflow.keras.applications import DenseNet121, DenseNet201
from tensorflow.keras.applications.densenet import preprocess_input

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.utils.np_utils import to_categorical
from tensorflow.keras.datasets import cifar100, cifar10
import warnings
warnings.filterwarnings("ignore")

print("Tensorflow-version:", tensorflow.__version__)

Tensorflow-version: 2.3.0


In [4]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data('fine')

# One-hot encoding for 100 classes.
y_train = to_categorical(y_train, 100)
y_test = to_categorical(y_test, 100)

print('Train: X=%s, y=%s' % (x_train.shape, y_train.shape))
print('Test: X=%s, y=%s' % (x_test.shape, y_test.shape))

Train: X=(50000, 32, 32, 3), y=(50000, 100)
Test: X=(10000, 32, 32, 3), y=(10000, 100)


In [7]:
input_shape=(32,32,3)
model_d=DenseNet201(weights='imagenet',include_top=False, input_shape=input_shape) 

x=model_d.output

x= BatchNormalization()(x)
x= Activation('relu')(x)
x= GlobalAveragePooling2D()(x)

# x= GlobalAveragePooling2D()(x)
# x= BatchNormalization()(x)
# x= Dropout(0.5)(x)
# x= Dense(1024,activation='relu')(x) 
# x= Dense(512,activation='relu')(x) 
# x= BatchNormalization()(x)
# x= Dropout(0.5)(x)

preds=Dense(100,activation='softmax')(x) #FC-layer

model=Model(inputs=model_d.input, outputs=preds)
model.summary()

#tf.keras.utils.plot_model( model , show_shapes=True )

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

batch_size = 128
epochs = 100

#(xtrain,xtest,ytrain,ytest)=train_test_split(x_train,y_train,test_size=0.4,random_state=42)
#print(xtrain.shape, xtest.shape)

history = model.fit( x_train , y_train , epochs=epochs , batch_size=batch_size )

#model.save("m_densenet121.h5")

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
zero_padding2d_2 (ZeroPadding2D (None, 38, 38, 3)    0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 16, 16, 64)   9408        zero_padding2d_2[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1/conv[0][0]                 
_______________________________________________________________________________________

In [8]:
results = model.evaluate(x_test, y_test, batch_size=batch_size)
print( 'Loss = {} and Accuracy = {} %'.format( results[0] , results[1] * 100 ) )


79/79 [==============================] - 3s 38ms/step - loss: 3.9518 - accuracy: 0.5625
Loss = 3.9517664909362793 and Accuracy = 56.25 %


## Teset with Image generator

In [ ]:
anne = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=5, verbose=1, min_lr=1e-3)
checkpoint = ModelCheckpoint('model.h5', verbose=1, save_best_only=True)

datagen = ImageDataGenerator(zoom_range = 0.2, horizontal_flip=True, shear_range=0.2)


datagen.fit(xtrain)
# Fits-the-model
history = model.fit(datagen.flow(xtrain, ytrain, batch_size=128),
               steps_per_epoch=xtrain.shape[0] // 128,
               epochs=50,
               verbose=2,
               callbacks=[anne, checkpoint],
               validation_data=(xtrain, ytrain))

In [ ]:
ypred = model.predict(xtest)

total = 0
accurate = 0
accurateindex = []
wrongindex = []

for i in range(len(ypred)):
    if np.argmax(ypred[i]) == np.argmax(ytest[i]):
        accurate += 1
        accurateindex.append(i)
    else:
        wrongindex.append(i)
        
    total += 1
    
print('Total-test-data;', total, '\taccurately-predicted-data:', accurate, '\t wrongly-predicted-data: ', total - accurate)
print('Accuracy:', round(accurate/total*100, 3), '%')

### heat map

In [ ]:
Ypred = model.predict(xtest)

Ypred = np.argmax(Ypred, axis=1)
Ytrue = np.argmax(ytest, axis=1)

cm = confusion_matrix(Ytrue, Ypred)
plt.figure(figsize=(12, 12))
ax = sns.heatmap(cm, cmap="rocket_r", fmt=".01f",annot_kws={'size':16}, annot=True, square=True, xticklabels=label, yticklabels=label)
ax.set_ylabel('Actual', fontsize=20)
ax.set_xlabel('Predicted', fontsize=20)